### Semana 5, Día 4

## AutoGen Core - Distribuido

¡Solo os daré un adelanto!

En parte porque no estoy seguro de su relevancia. Si deseas que agregue más contenido, por favor, házmelo saber.

In [2]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
# from langchain.agents import Tool
from langchain_core.tools import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Comienza con nuestra clase de Mensaje

In [3]:

@dataclass
class Message:
    content: str

### Y ahora: un host para nuestro entorno de ejecución distribuido

In [5]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Vamos a reintroducir una herramienta

In [6]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Útil para cuando necesitas buscar en Internet.")
autogen_serper = LangChainToolAdapter(langchain_serper)

ValidationError: 1 validation error for GoogleSerperAPIWrapper
  Value error, Did not find serper_api_key, please add an environment variable `SERPER_API_KEY` which contains it, or pass `serper_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [5]:
instruction1 = "Para ayudar con una decisión sobre si usar AutoGen en un nuevo proyecto de Agentes de IA, \
por favor investiga y responde brevemente con razones a favor de elegir AutoGen; los pros de AutoGen."

instruction2 = "Para ayudar con una decisión sobre si usar AutoGen en un nuevo proyecto de Agentes de IA, \
por favor investiga y responde brevemente con razones en contra de elegir AutoGen; los contras de Autogen."

judge = "Debes tomar una decisión sobre si usar AutoGen para un proyecto. \
Tu equipo de investigación ha llegado a las siguientes razones a favor y en contra. \
Basado puramente en la investigación de tu equipo, por favor responde con tu decisión y una breve justificación."

### Y hacer algunos Agentes

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros de AutoGen:\n{response1.content}\n\n## Cons de AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Responde con tu decisión y una breve justificación"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decisión:\n\n" + response.chat_message.content)


In [8]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [11]:
response = await worker.send_message(Message(content="¡Empezamos!"), agent_id)

In [12]:
display(Markdown(response.content))

## Pros de AutoGen:
Aquí tienes algunas razones a favor de usar AutoGen en proyectos de Agentes de IA:

1. **Facilidad de uso**: AutoGen ofrece un marco fácil de usar que permite a los desarrolladores implementar agentes de forma eficiente sin complicaciones innecesarias.

2. **Capacidades avanzadas**: Es ideal para la automatización compleja y entornos dinámicos, lo que permite a los agentes gestionar tareas diversas y ajustarse a diferentes escenarios de trabajo.

3. **Alta personalización**: Los usuarios pueden personalizar los agentes con un control detallado sobre sus componentes y flujos de trabajo, adaptándolos a necesidades específicas del proyecto.

4. **Escalabilidad**: Su arquitectura escalable permite la integración de múltiples agentes, cada uno con roles y capacidades específicos, fomentando la colaboración y mejorando la resolución de problemas.

5. **Integración con múltiples herramientas**: AutoGen permite la integración con diversos modelos de lenguaje y herramientas, lo que amplía sus aplicaciones y capacidades.

Estas ventajas hacen de AutoGen una opción fuerte para proyectos que requieren implementación de Agentes de IA.

TERMINATE

## Cons de AutoGen:
Aquí tienes algunas desventajas de usar AutoGen para un nuevo proyecto de Agentes de IA:

1. **Capacidades de IA Limitadas**: AutoGen puede no ser la mejor opción para tareas de automatización complejas que requieren un alto nivel de inteligencia artificial, limitando su aplicabilidad en escenarios más avanzados.

2. **Menos Personalizable**: Aunque permite cierto grado de personalización, puede haber limitaciones en comparación con otros frameworks que ofrecen más flexibilidad en la integración y configuración de diversos componentes.

3. **Dependencia de API**: El uso de AutoGen normalmente requiere claves de API para su integración con modelos de lenguaje, lo que puede incorporar costos adicionales y complicar la gestión de acceso y autorización.

4. **Curva de Aprendizaje**: La utilización de AutoGen puede requerir que el equipo de desarrollo aprenda a manejar nuevas herramientas y paradigmas, lo que podría ralentizar el progreso inicial del proyecto.

5. **Menos soporte en la automatización entre agentes**: Si bien soporta un rango de tipos de agentes, puede ser menos eficaz en facilitar interacciones complejas entre estos, limitando su utilidad en proyectos que requieren un alto grado de colaboración entre agentes.

Considerar estas desventajas te ayudará a tomar una decisión informada sobre el uso de AutoGen en tu proyecto. 

TERMINATE



## Decisión:

Después de analizar los pros y los contras de AutoGen, mi decisión es **utilizar AutoGen para el proyecto de Agentes de IA**.

Justificación: A pesar de las preocupaciones sobre las capacidades de IA limitadas y la curva de aprendizaje, las ventajas de facilidad de uso, capacidades avanzadas y alta personalización pesan considerablemente. La escalabilidad de AutoGen y su capacidad para integrarse con múltiples herramientas lo convierten en una opción robusta para entornos dinámicos donde la eficiencia en la implementación y la adaptabilidad son esenciales. Además, su capacidad para manejar tareas diversas puede ser muy valiosa en este proyecto. Las desventajas pueden ser gestionadas con un enfoque estratégico de desarrollo y formación del equipo. 

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()